# Seminar: Gradient Boosting
Course: Machine Learning by professor Evgeny Burnaev
<br>
Author: Andrey Lange

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor

## Example 1: Gradient Boosting for regression

In [ ]:
def get_dataset_1d():
    # prepare dataset
    n = 1                      # number of features
    N = 100**n                 # number of samples
    np.random.seed(0)
    X = np.random.random((N, n))*3
    coeffs = 1 +  2 * np.random.random((n, 1))
    y = np.sin(np.matmul(X*X, coeffs)) + np.random.random((N, 1))/3
    y = y.ravel()
    
    return (X, y)
    
def plot_results(X, y, y_pred, title=''):
    plt.plot(X, y, '*b')
    plt.plot(X, y_pred, '.r')
    plt.title(title)
    plt.xlabel('x1')
    plt.ylabel('y')
    plt.show()


---

### Question 1.
In the following example a Gradient Boosting regression is performed by only 1 tree and very small shrinkage `nu=1e-10`. The solution looks like a constant.

1.1. Why it looks like a constant?
<br>
1.2. Try to find that constant value taking into account that `loss='ls'` and draw it on the same plot.

In [ ]:
X, y = get_dataset_1d()

clf = GradientBoostingRegressor(loss='ls', max_depth=1, learning_rate=1e-10, n_estimators=1)
clf.fit(X, y)
y_pred = clf.predict(X)
plot_results(X, y, y_pred)

### Solution 1.
1.1. In case of only 1 tree the final boosting is
$$f_1(x) = f_0(x)+\nu \sum\limits_{j=1}^{J} \gamma_j I(x\in R_j).$$
Since $\nu$ is too small, $f_1(x)\approx f_0(x)$ and $f_0(x)$ is the initialization value
$$f_0(x) = {\rm argmin}_{\gamma} \sum_{i=1}^{N} L(y, \gamma).$$

1.2.
For the loss function $L(y, \gamma) = (y - \gamma)^2/2$ it is the mean of targets:
$$ f_0(x) = \frac{1}{N}\sum_{i=1}^N y_i.$$
You can add 
```
plt.plot(X, [np.mean(y)]*len(y), 'ok')
```
before `plot_results()` call.


---

### Question 2. 
Solve above question for `loss='lad'`.

### Solution 2. 
Change `np.mean()` for `np.median()`.


---

### Question 3. 
Some managers of industrial Data Science projects said me that each tree in Gradient Boosting is fit to the targets that are simply the differences $y_i-f_{m-1}(x_i)$ between the values $y_i$ and the current approximation $f_{m-1}(x_i)$ found on the previous step $m-1$. When is it correct?

### Solution 3.
It is true in case of Gradient Boosting for regression with the squared error loss 
$$L(y_i, f(x_i)) = \frac{1}{2}[y_i-f(x_i)]^2.$$


The gradients
$$ r_{im} = - \left[\frac{\partial L(y_i, f(x_i))}{\partial f(x_i)}\right]_{f=f_{m-1}}$$
are calculated for each data point $(x_i, y_i)$ on every step $m$. Then a regression tree is fit to the targets $r_{im}$. For the squared error loss the negative gradient is exactly the difference 
$$r_{im} = y_i-f_{m-1}(x_i).$$


---

### Question 4*. 
Actually, the managers from the question above said not only to what the trees are fit but also that the current solution is updated by simply adding each new tree multiplied by a shrinkage parameter and ignore any sophisticated math! :[]

The following code shows 2 ways of using the Gradient Boosting for regression:
1. Using the class `GradientBoostingRegressor()` - as it should be done if you use scikit-learn
2. Our own implementation by adding a tree step by step with shrinkage

Ensure that the pictures are the same no matter how you change the hyperparameters `max_depth`, `n_estimators` and `nu`!

In our implementation we fit each tree to the targets $y_i-f_{m-1}(x_i)$ (if you answered on the above question you know that it is correct here) multiply by shrinkage $\nu$ and add to the current model. But it seems that something is missing in our code. No matter how exactly the Gradient Boosting is implemented in scikit-learn, there are many slightly different variants. Assume that it is as described here https://scikit-learn.org/stable/modules/ensemble.html#mathematical-formulation, where the steepest descent chooses the optimal step length
$$
\gamma_m = \arg\min_{\gamma} \sum_{i=1}^{n} 
L
\left(
y_i, f_{m-1}(x_i)
- \gamma \nabla_F L(y_i, f_{m-1}(x_i))
\right),
$$
which is used for the model update
$$
f_m(x) = f_{m-1}(x) - \gamma_m \sum_{i=1}^{n} \nabla_F L(y_i, f_{m-1}(x_i)).
$$
But we did not implement it! Justify the correctness of our easy implementation of GB.

In [ ]:
X, y = get_dataset_1d()

max_depth = 3
n_estimators = 6
nu = 0.8                            # shrinkage

# usual Gradient Boosting call
clf = GradientBoostingRegressor(loss='ls', max_depth=max_depth, learning_rate=nu, n_estimators=n_estimators)
clf.fit(X, y)
f = clf.predict(X)
plot_results(X, y, f, 'Gradient Boosting from scikit-learn')

# my Gradient Boosting implementation
clf = DecisionTreeRegressor(max_depth=max_depth)
f = np.mean(y)                      # initialization
for m in range(1, n_estimators+1):
    f = f + clf.fit(X, y - f).predict(X) * nu # fit to the difference and shrink
plot_results(X, y, f, 'My simple Gradient Boosting, the same as above!')

### Solution 4*.

For the squared error loss $L(y_i, f(x_i)) = \frac{1}{2}[y_i-f(x_i)]^2$ we have
$$ L\left(y_i, f_{m-1}(x_i) - \gamma \nabla_F L(y_i, f_{m-1}(x_i))\right) =
 \frac{1}{2} (1 - \gamma)^2 (y_i - f_{m-1}(x_i))^2,
$$
and thus $\gamma_m=1$ or in other words we can skip the steepest descent in this case.

## Example 2: Gradient Boosting for classification, CatBoost.

Lets consider Titanic data! Below is yet another example of easy feature engineering, data preprocessing and the application of Gradient Boosting

In [ ]:
## load data and some easy preprocessing, even Feature Engineering
# we use only training dataset because the Kaggle's testing one does not have class labels 
# and so we can not measure the model quality

X_train = pd.read_csv('https://raw.githubusercontent.com/adasegroup/ML2020_seminars/master/seminar9/data/train.csv')
y_train = X_train['Survived']
X_train = X_train.drop(['PassengerId', 'Survived', 'Name', 'Ticket'], axis=1)

# take only the first letter from the Cabin number, which is maybe the ship level...
def keep_only_level_of_cabin(X):
    idx = X['Cabin'].notnull()
    X.loc[idx, 'Cabin'] = [s.strip()[0] for s in X['Cabin'][idx].values]
    return X

# CatBoost can not process categorical features with NaN, we set them to the string 'MISS'
def change_NaN_to_str(X, cat_features_cols):
    for col in cat_features_cols:
        idx = X[col].isnull()
        X.loc[idx, col] = 'MISS'
    return X

X_train = keep_only_level_of_cabin(X_train)
X_train = change_NaN_to_str(X_train, ['Sex', 'Cabin', 'Embarked'])

In [ ]:
X_train

### Note: 

  1. We use categorical features and CatBoost processes it ok.

  
  2. Below you can see the tuning of CatBoost with cross-validation, you can play with the hyperparameter ranges.
  
  
  3. You can "unnatural disable" cross-validation and ensure that the best score increases (although it is impractical because of overfitting).
  
  
  4. CatBoost processes contineous feature 'Age' despite there are NaNs. It is easy for trees - we can, say, just ignore these values.
  
  
  5. We preprocess the 'Cabin' feature so that it has only the level ('C', 'E', ...). 
  This is so that there would not be a large number of unique noninformative values. And here we process NaNs as special 'MISS' value because it may mean not the incompletness in the data, but that no any proper value can be for some samples (it is highly correlated with 'Pclass' feature).  
  
  
  6. You can play with feature importances: drop the most important feature and check how the accuracy and the roc_auc reduce. Do the same with the least important feature and feel who affects more. 

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve

clf = CatBoostClassifier(loss_function = 'Logloss',
                         # categorical features processing: 
                         cat_features  = np.where(X_train.columns.isin(['Sex', 'Cabin', 'Embarked']))[0], 
                         verbose = 0)

clf = GridSearchCV(clf,
                   # you can play with tuning, up to your CPU performance:
                   {'max_depth': [1, 3, 5], 'n_estimators': [50, 150], 'learning_rate': [0.001, 0.01, 0.1, 1]},
                   # cross-validation prevents overfitting! you can disable it by setting 
                   # cv = [(np.array(range(len(y_train))), np.array(range(len(y_train))))]
                   cv = 3)
clf.fit(X_train, y_train)

print('best accuracy score:', clf.best_score_) # validation score

# feature importances
fi = pd.Series(clf.best_estimator_.feature_importances_, index=X_train.columns)
fi.sort_values(ascending=False).plot(kind='bar')
plt.title('feature importances')
plt.show()

# now let's draw different ROC curves
plt.figure(figsize=[9, 6])
fpr, tpr, _ = roc_curve(y_train, clf.predict_proba(X_train)[:, 1])
plt.plot(fpr, tpr, 'r', label='train')
fpr, tpr, _ = roc_curve(y_train, clf.predict(X_train))
plt.plot(fpr, tpr, '--o', label='binary output, train')
plt.legend(bbox_to_anchor=(0.999, 1))
plt.title('ROC curves')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.show()